In [2]:
# packages 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math

import os
import xarray as xr
from glob import glob

import netCDF4 as nc

In [7]:
# FIND LOCATION FOR DATA:
# open predictor data:
pres_path = '/Users/rosiekrasnoff/Desktop/SWE_final_project/predictor_data/ps_watch-wfdei_2001_2010.nc4'
hum_path = '/Users/rosiekrasnoff/Desktop/SWE_final_project/predictor_data/rhs_watch-wfdei_2001_2010.nc4'
temp_path = '/Users/rosiekrasnoff/Desktop/SWE_final_project/predictor_data/tas_watch-wfdei_2001_2010.nc4'

pres_ds = xr.open_dataset(pres_path)
#pres_df = pres_ds.to_dataframe()
#print(pres_ds)

lats = pres_ds.lat
lons = pres_ds.lon
lat_p = lats.values
lon_p = lons.values

for i in range(len(lat_p)):
    for j in range(len(lon_p)):
        if math.isnan(lat_p[i]) == 0:
            if lat_p[i] >= 42 and lat_p[i] <= 43 and lon_p[j] <= -75 and lon_p[j] >= -77:
                print(i,j,lat_p[i], lon_p[j])
                
# results: use i = 95 for lat = 42.25 and j = 206 for lon = -76.75

94 206 42.75 -76.75
94 207 42.75 -76.25
94 208 42.75 -75.75
94 209 42.75 -75.25
95 206 42.25 -76.75
95 207 42.25 -76.25
95 208 42.25 -75.75
95 209 42.25 -75.25


In [6]:
#Finding the correct lat and lon indexes from an older version of the GlobSnow files:
fn1 = '/Users/rosiekrasnoff/Desktop/SWE_final_project/GlobSnow_SWE_L3A_19810101_v2.0.nc'

input_ps = xr.open_dataset(fn1)
#print(input_ps)
lats = input_ps.lat
lons = input_ps.lon
lat_np = lats.values
lon_np = lons.values

# print the lat and lon values for those close to desired location: 42.4 N and -76.5 E
for x in range(720):
    for y in range(720):
        if math.isnan(lat_np[x,y]) == 0:
            #print(lat_np[x,y])
            #if lat_np[x,y] >= 42.2 and lat_np[x,y] <= 42.6 and lon_np[x,y] <= -76.3 and lon_np[x,y] >= -76.7:
            if lat_np[x,y] >= 41 and lat_np[x,y] <= 42 and lon_np[x,y] <= -75 and lon_np[x,y] >= -77:

    
                print(x,y,lat_np[x,y], lon_np[x,y])
                
# results: use x = 312, y = 160

306 157 41.18314 -75.10372
306 158 41.42222 -75.03328
307 156 41.0066 -75.43631
307 157 41.24622 -75.36764
307 158 41.48553 -75.29833
307 159 41.72455 -75.22838
307 160 41.96326 -75.15777
308 156 41.06833 -75.69972
308 157 41.30818 -75.63218
308 158 41.54772 -75.56402
308 159 41.78697 -75.49522
309 156 41.12895 -75.96375
309 157 41.36901 -75.89736
309 158 41.60878 -75.83035
309 159 41.84826 -75.76272
310 156 41.18844 -76.2284
310 157 41.42873 -76.16316
310 158 41.66872 -76.09731
310 159 41.90841 -76.03084
311 155 41.00601 -76.55713
311 156 41.24681 -76.49364
311 157 41.48731 -76.42956
311 158 41.72752 -76.36489
311 159 41.96744 -76.2996
312 155 41.06304 -76.82181
312 156 41.30404 -76.75948
312 157 41.54476 -76.69657
312 158 41.78518 -76.63307
313 157 41.60107 -76.96416
313 158 41.84171 -76.90185


In [48]:
# create an array that includes data at selected single location:
# [date   SWE   temp   humd   airpress]

i = 95 # index for correct lat
j = 206 # index for correct lon
x = 312 # index for correct lat in SWE
y = 160 # index for correct lon in SWE

# predictor datasets already opened above
temp_ds = xr.open_dataset(temp_path) # temperature dataset
humd_ds = xr.open_dataset(hum_path)  # humidity dataset
pres_ds = xr.open_dataset(pres_path) # air pressure dataset

date = pd.DatetimeIndex(temp_ds.time)

#length = (len(temp_ds.time))
#print(length)

df = pd.DataFrame(columns=['date','SWE','temp','hum','pres'])

# loop through time in predictor files, t is time index, one SWE file (may) exist for every day 
for t in range((len(temp_ds.time))):
    
    # date into form yyyymmdd for name of SWE file
    dyear = (date.year[t])
    dmonth = (date.month[t])
    dday = (date.day[t])
    t_date = dyear*10000+dmonth*100+dday 
    str_date = str(t_date)
    
    
    if dmonth < 6 or dmonth > 8:
        #print(str_date)
        
        # open SWE data if it exists:
        filename = str_date + '_northern_hemisphere_swe_0.25grid.nc'
        swe_path = '/Users/rosiekrasnoff/Desktop/SWE_final_project/GlobSnow_data/' + filename
    
        try: #some SWE files don't exist, so try to open
            swe_ds = xr.open_dataset(swe_path) #swe dataset
            swe_numpy = swe_ds.swe.values #swe values in numpy array
            my_swe = swe_numpy[x,y]
            my_temp = temp_ds.tas[t,i,j].values.item()
            my_humd = humd_ds.hurs[t,i,j].values.item()
            my_pres = pres_ds.ps[t,i,j].values.item()
            
            d = {'date':date[t],'SWE':my_swe,'temp':my_temp,'hum':my_humd,'pres':my_pres}
            df = df.append(d, ignore_index=True)
        except:
            pass

/var/folders/vk/73wdxc_14wj23yn81gyq7zcr0000gn/T/ipykernel_75557/1568913556.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(d, ignore_index=True)
/var/folders/vk/73wdxc_14wj23yn81gyq7zcr0000gn/T/ipykernel_75557/1568913556.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(d, ignore_index=True)
/var/folders/vk/73wdxc_14wj23yn81gyq7zcr0000gn/T/ipykernel_75557/1568913556.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(d, ignore_index=True)
/var/folders/vk/73wdxc_14wj23yn81gyq7zcr0000gn/T/ipykernel_75557/1568913556.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(d,

/var/folders/vk/73wdxc_14wj23yn81gyq7zcr0000gn/T/ipykernel_75557/1568913556.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(d, ignore_index=True)
/var/folders/vk/73wdxc_14wj23yn81gyq7zcr0000gn/T/ipykernel_75557/1568913556.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(d, ignore_index=True)
/var/folders/vk/73wdxc_14wj23yn81gyq7zcr0000gn/T/ipykernel_75557/1568913556.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(d, ignore_index=True)
/var/folders/vk/73wdxc_14wj23yn81gyq7zcr0000gn/T/ipykernel_75557/1568913556.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(d,

/var/folders/vk/73wdxc_14wj23yn81gyq7zcr0000gn/T/ipykernel_75557/1568913556.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(d, ignore_index=True)
/var/folders/vk/73wdxc_14wj23yn81gyq7zcr0000gn/T/ipykernel_75557/1568913556.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(d, ignore_index=True)
/var/folders/vk/73wdxc_14wj23yn81gyq7zcr0000gn/T/ipykernel_75557/1568913556.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(d, ignore_index=True)
/var/folders/vk/73wdxc_14wj23yn81gyq7zcr0000gn/T/ipykernel_75557/1568913556.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(d,

/var/folders/vk/73wdxc_14wj23yn81gyq7zcr0000gn/T/ipykernel_75557/1568913556.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(d, ignore_index=True)
/var/folders/vk/73wdxc_14wj23yn81gyq7zcr0000gn/T/ipykernel_75557/1568913556.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(d, ignore_index=True)
/var/folders/vk/73wdxc_14wj23yn81gyq7zcr0000gn/T/ipykernel_75557/1568913556.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(d, ignore_index=True)
/var/folders/vk/73wdxc_14wj23yn81gyq7zcr0000gn/T/ipykernel_75557/1568913556.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(d,

/var/folders/vk/73wdxc_14wj23yn81gyq7zcr0000gn/T/ipykernel_75557/1568913556.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(d, ignore_index=True)
/var/folders/vk/73wdxc_14wj23yn81gyq7zcr0000gn/T/ipykernel_75557/1568913556.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(d, ignore_index=True)
/var/folders/vk/73wdxc_14wj23yn81gyq7zcr0000gn/T/ipykernel_75557/1568913556.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(d, ignore_index=True)
/var/folders/vk/73wdxc_14wj23yn81gyq7zcr0000gn/T/ipykernel_75557/1568913556.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(d,

...

/var/folders/vk/73wdxc_14wj23yn81gyq7zcr0000gn/T/ipykernel_75557/1568913556.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(d, ignore_index=True)
/var/folders/vk/73wdxc_14wj23yn81gyq7zcr0000gn/T/ipykernel_75557/1568913556.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(d, ignore_index=True)
/var/folders/vk/73wdxc_14wj23yn81gyq7zcr0000gn/T/ipykernel_75557/1568913556.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(d, ignore_index=True)
/var/folders/vk/73wdxc_14wj23yn81gyq7zcr0000gn/T/ipykernel_75557/1568913556.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(d,

/var/folders/vk/73wdxc_14wj23yn81gyq7zcr0000gn/T/ipykernel_75557/1568913556.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(d, ignore_index=True)
/var/folders/vk/73wdxc_14wj23yn81gyq7zcr0000gn/T/ipykernel_75557/1568913556.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(d, ignore_index=True)
/var/folders/vk/73wdxc_14wj23yn81gyq7zcr0000gn/T/ipykernel_75557/1568913556.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(d, ignore_index=True)
/var/folders/vk/73wdxc_14wj23yn81gyq7zcr0000gn/T/ipykernel_75557/1568913556.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(d,

/var/folders/vk/73wdxc_14wj23yn81gyq7zcr0000gn/T/ipykernel_75557/1568913556.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(d, ignore_index=True)
/var/folders/vk/73wdxc_14wj23yn81gyq7zcr0000gn/T/ipykernel_75557/1568913556.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(d, ignore_index=True)
/var/folders/vk/73wdxc_14wj23yn81gyq7zcr0000gn/T/ipykernel_75557/1568913556.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(d, ignore_index=True)
/var/folders/vk/73wdxc_14wj23yn81gyq7zcr0000gn/T/ipykernel_75557/1568913556.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(d,

/var/folders/vk/73wdxc_14wj23yn81gyq7zcr0000gn/T/ipykernel_75557/1568913556.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(d, ignore_index=True)
/var/folders/vk/73wdxc_14wj23yn81gyq7zcr0000gn/T/ipykernel_75557/1568913556.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(d, ignore_index=True)
/var/folders/vk/73wdxc_14wj23yn81gyq7zcr0000gn/T/ipykernel_75557/1568913556.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(d, ignore_index=True)
/var/folders/vk/73wdxc_14wj23yn81gyq7zcr0000gn/T/ipykernel_75557/1568913556.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(d,

           date  SWE        temp        hum          pres
0    2001-01-01  0.0  263.546143  73.508492  96964.562500
1    2001-01-02  0.0  262.069763  76.259216  97539.937500
2    2001-01-03  0.0  264.118225  73.916405  97338.460938
3    2001-01-04  0.0  266.926605  69.559303  96404.531250
4    2001-01-05  0.0  266.665710  82.092102  95651.828125
...         ...  ...         ...        ...           ...
2081 2010-12-27  0.0  263.790100  76.103828  95636.750000
2082 2010-12-28  0.0  268.415771  67.402588  96249.492188
2083 2010-12-29  0.0  270.088165  72.630386  96626.937500
2084 2010-12-30  0.0  269.368835  75.820518  97076.250000
2085 2010-12-31  0.0  276.260925  75.788727  96993.101562

[2086 rows x 5 columns]


In [53]:
print(df)

           date  SWE        temp        hum          pres
0    2001-01-01  0.0  263.546143  73.508492  96964.562500
1    2001-01-02  0.0  262.069763  76.259216  97539.937500
2    2001-01-03  0.0  264.118225  73.916405  97338.460938
3    2001-01-04  0.0  266.926605  69.559303  96404.531250
4    2001-01-05  0.0  266.665710  82.092102  95651.828125
...         ...  ...         ...        ...           ...
2081 2010-12-27  0.0  263.790100  76.103828  95636.750000
2082 2010-12-28  0.0  268.415771  67.402588  96249.492188
2083 2010-12-29  0.0  270.088165  72.630386  96626.937500
2084 2010-12-30  0.0  269.368835  75.820518  97076.250000
2085 2010-12-31  0.0  276.260925  75.788727  96993.101562

[2086 rows x 5 columns]


In [55]:
x = df['SWE']
print(x)

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
2081    0.0
2082    0.0
2083    0.0
2084    0.0
2085    0.0
Name: SWE, Length: 2086, dtype: float64
